# Authors:
- Maria Musiał 156062
-  Martyna Stasiak 156071

Problem representation

Initialize matrix

In [56]:
import numpy as np
import pulp as pl

# M = float('inf')
M = 999999

costs = np.array([[7, 5, 5, 0],
                 [3, 10, 10, M],
                 [3, 10, 10, 0],
                 [M, M, 0, 0]])
supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])

## Use NorthWest method to find our path

In [57]:
def north_west_corner(supply, demand):
    """
    North West Corner Method for finding path (initial solutions) of transportation problem
    """
    m = len(supply) 
    n = len(demand)    #m - number of suppliers, n- number of consumers
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    path = []
    while len(path) < m + n - 1:  #m+n basic variables
        s = supply_copy[i]   
        d = demand_copy[j]
        v = min(s, d)  #get maximum possible value (minimum of s and d)
        supply_copy[i] -= v   #delete value from supply
        demand_copy[j] -= v   #delete value from demand
        path.append(((i, j), v))   #add to our path
        if supply_copy[i] == 0 and i < m - 1:  #we cant use this supplier anymore
            i += 1
        elif demand_copy[j] == 0 and j < n - 1:   #we cant use this consumer anymore
            j += 1
    return path


supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])
path = north_west_corner(supply, demand)
print(path)

[((0, 0), 30), ((1, 0), 10), ((1, 1), 10), ((2, 1), 30), ((2, 2), 20), ((2, 3), 30), ((3, 3), 80)]


## Get U collumn and V row 

In [58]:
def get_us_and_vs(path, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    
    #set ui=0 of the row that is assigned the most
    row_common = np.array([value[0][0] for value in path])
    unique, counts = np.unique(row_common, return_counts=True)
    most_common_row = unique[np.argmax(counts)]
    us[most_common_row] = 0
    
    path_copy = path.copy()
    while len(path_copy) > 0:     #while we dont calculate for all basic variables
        for index, bv in enumerate(path_copy):
            i, j = bv[0]         #coordinates of basic variable
            if us[i] is None and vs[j] is None: continue      #skip cells that dont have ui or vi assigned yet
                
            cost = costs[i][j]
            if us[i] is None:          #assign ui based on vi
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]     #assign vi based on ui
            path_copy.pop(index)        #dont take the basic variable into account anymore
            break
            
    return us, vs      


path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)
print("U collumn: \n", us, "\nV row:\n", vs)   

U collumn: 
 [4, 0, 0, 0] 
V row:
 [3, 10, 10, 0]


## Calculate the not basic variable values

In [59]:
def get_Cj(path, costs, us, vs):
    Cj = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in path])
            if non_basic:
                Cj.append(((i, j), cost - us[i] - vs[j] ))
    
    return Cj

path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)

get_Cj(path, costs, us, vs)

[((0, 1), -9),
 ((0, 2), -9),
 ((0, 3), -4),
 ((1, 2), 0),
 ((1, 3), 999999),
 ((2, 0), 0),
 ((3, 0), 999996),
 ((3, 1), 999989),
 ((3, 2), -10)]

## Check if we need another iteration (if we have coefficient that is negative )

In [60]:
def improvement(Cj):
    for p, v in Cj:
        if v < 0:
            return True
    return False
print(improvement(get_Cj(path, costs, us, vs)))

True


## If true, check position of entering variable (smallest value)

In [61]:
def get_entering_variable(Cj):
    return min(Cj, key=lambda x: x[1])[0]

entering_var = get_entering_variable(get_Cj(path, costs, us, vs))
print("Position of entering variable: ", entering_var)


Position of entering variable:  (3, 2)


## Chain rule of finding leaving variable and changing values

In [62]:
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    row_candidates = [n for n in not_visited if n[0] == last_node[0]]
    column_candidates = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return row_candidates + column_candidates
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
    if row_move: return column_candidates
    return row_candidates

In [63]:
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])


path = north_west_corner(supply, demand)
us, vs = get_us_and_vs(path, costs)
Cj = get_Cj(path, costs, us, vs)
improvement(Cj)

entering_var = get_entering_variable(Cj)
loop = get_loop([p for p, v in path], entering_var)
print("loop of donors and recipients ", loop)

loop of donors and recipients  [(3, 2), (3, 3), (2, 3), (2, 2)]


In [64]:
def ChainRule(path, loop):
    path_copy = path.copy()
    donors = loop[1::2]
    recipients = loop[::2]
    
    #Check minimum value in the donors in the loop
    donors_path = [(coords, value) for coords, value in path_copy if coords in donors]
    recipients_path = [(coords, value) for coords, value in path_copy if coords in recipients]
    leaving_var, min_value = min(donors_path, key=lambda x: x[1])
        
    #Add the entering variable to the path
    path_copy.append((loop[0], 0))
    #Update the path values
    for i, ((x,y),z) in enumerate(path_copy):
        if (x,y) in recipients:
            path_copy[i] = ((x,y), z + min_value)
        if (x,y) in donors:
            path_copy[i] = ((x,y), z - min_value)
    #Remove the leaving variable 
    path_copy.remove((leaving_var, 0))
    
    return path_copy

print("New path:\n", ChainRule(path, loop))

New path:
 [((0, 0), 30), ((1, 0), 10), ((1, 1), 10), ((2, 1), 30), ((2, 3), 50), ((3, 3), 60), ((3, 2), 20)]


## Final function for transportation method

In [65]:
def transportation_method(costs, supply, demand):
    
    def iteration(path, num_iter, chainsize):
        num_iter += 1
        us, vs = get_us_and_vs(path, costs)
        Cj = get_Cj(path, costs, us, vs)
        if improvement(Cj):
            entering = get_entering_variable(Cj)
            loop = get_loop([p for p, v in path], entering)
            chainsize.append(len(loop))
            return iteration(ChainRule(path, loop), num_iter, chainsize)
        return path, num_iter, chainsize
    
    path = north_west_corner(supply, demand)
    
    final_basic_variables, num_iter, chainsizes = iteration(path, 0, [])
    
    matrix = np.zeros((len(costs), len(costs[0])))
    Z = 0
    for (i, j), v in final_basic_variables:
        matrix[i][j] = v
        Z += (v * costs[i][j])
    
    if len(chainsizes) > 0:
        avg_chaisize = sum(chainsizes) / len(chainsizes)

    return matrix, Z, num_iter, avg_chaisize


# M = float('inf')
M = 999999
costs = np.array([[7, 5, 5, 0],
                 [3, 10, 10, M],
                 [3, 10, 10, 0],
                 [M, M, 0, 0]])
supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])

matrix, Z, num_iter, avg_chaisize = transportation_method(costs, supply, demand)
print("Final matrix:\n", matrix)
print("Final objective function Z: ", Z)
print(f"Done in {num_iter} iterations")
print("Chainsizes: \n", avg_chaisize)

Final matrix:
 [[ 0. 30.  0.  0.]
 [20.  0.  0.  0.]
 [20. 10.  0. 50.]
 [ 0.  0. 20. 60.]]
Final objective function Z:  370
Done in 4 iterations
Chainsizes: 
 4.0


## Comparison with Oracle

In [66]:
def verify_with_pulp(cost, supply, demand):
    prob = pl.LpProblem("Transportation_Problem", pl.LpMinimize)
    rows, cols = cost.shape
    x = pl.LpVariable.dicts("x", (range(rows), range(cols)), 0, None, pl.LpInteger)

    prob += pl.lpSum(cost[i][j] * x[i][j] for i in range(rows) for j in range(cols))

    for i in range(rows):
        prob += pl.lpSum(x[i][j] for j in range(cols)) == supply[i]
    for j in range(cols):
        prob += pl.lpSum(x[i][j] for i in range(rows)) == demand[j]

    prob.solve()

    solution = np.zeros_like(cost, dtype=int)
    for i in range(rows):
        for j in range(cols):
            solution[i, j] = int(x[i][j].varValue)

    return solution, pl.value(prob.objective)

pulp_solution, pulp_cost = verify_with_pulp(costs, supply, demand)
print("PuLP solution:\n", pulp_solution)
print("PuLP objective value:", pulp_cost)

PuLP solution:
 [[ 0 30  0  0]
 [10 10  0  0]
 [30  0  0 50]
 [ 0  0 20 60]]
PuLP objective value: 370.0


## Analyzing model performance

In [ ]:
def generate_problem(size, val_range=(1, 10), M=999999):
    """Generate a transportation problem with special cases like zeroes, M-values, and zero-rows/columns."""
    # Random costs
    costs = np.random.randint(val_range[0], val_range[1], (size, size))

    # Introduce random zeroes
    num_zeroes = np.random.randint(1, size)  # Random number of zeroes
    for _ in range(num_zeroes):
        i, j = np.random.randint(0, size, 2)
        costs[i, j] = 0

    # Introduce random M-values
    num_m_values = np.random.randint(0, size//3)  # Random number of M-values
    for _ in range(num_m_values):
        i, j = np.random.randint(0, size, 2)
        costs[i, j] = M

    # Occasionally set a random row to zero (zero-row)
    if np.random.rand() < 0.5:  # 50% chance of having a zero-row
        row = np.random.randint(0, size)
        costs[row, :] = 0

    # Occasionally set a random column to zero (zero-column)
    if np.random.rand() < 0.5:  # 50% chance of having a zero-column
        col = np.random.randint(0, size)
        costs[:, col] = 0

    # Random supply and demand
    supply = np.random.randint(val_range[0], val_range[1], size)
    demand = np.random.randint(val_range[0], val_range[1], size)

    # Balance supply and demand
    total_supply = np.sum(supply)
    total_demand = np.sum(demand)
    if total_supply > total_demand:
        demand[-1] += (total_supply - total_demand)
    elif total_demand > total_supply:
        supply[-1] += (total_demand - total_supply)

    return costs, supply, demand



def analyze(sizes, runs=10):
    results = []
    for size in sizes:
        total_z = 0
        total_iter = 0
        total_avg_chain = 0
        total_p = 0
        
        for _ in range(runs):
            costs, supply, demand = generate_problem(size)
            _, Z, num_iters, avg_chain = transportation_method(costs, supply, demand)
            _, p_cost = verify_with_pulp(costs, supply, demand)
            total_avg_chain += avg_chain
            total_iter += num_iters
            total_z += Z
            total_p += p_cost
            
        avg_Z = total_z / runs
        avg_iter = total_iter / runs
        avg_chaisize = total_avg_chain / runs
        avg_p = total_p / runs
        results.append((size, avg_Z, avg_iter, avg_chaisize, avg_p))
        
    return results 


sizes = [5, 10, 15]
results = analyze(sizes, 10)
print("Average results over 10 runs: ")
print("Size\tObjective function\tPulp obj func\tIterations\tChain size")
for size, z, iters, chain, pulp_Z in results:
    print(f"{size:.2f}\t\t{z:.2f}\t\t  {pulp_Z:.2f}\t\t {iters:.2f}\t\t  {chain:.2f}")

Average results over 10 runs: 
Size	Objective function	Pulp obj func	Iterations	Chain size
5.00		53.68		  53.68		 6.41		  5.05
10.00		83.71		  83.71		 20.43		  7.77
15.00		105.10		  105.10		 39.91		  10.47


- We need to feed it some sensible matrix. The generator isnt sensible, as we have objective with 500000 at times
-> Construct the test sampels randomly (eg. cost, supply, demand=rand(1,10)), but let them occasionaly contain zeros, m-values, and even 0-rows and 0-collumns. total supply should == total demand

chainsize can be zero!
